In [1]:
# example 2: Drectly using the pdb API 
# example @ https://search.rcsb.org/#search-example-3

In [2]:
import requests
import json

In [3]:
seq = "TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVI\
RGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEI\
YQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG"
result_type = "polymer_entity" # data with alignments
#result_type = "entry" # pdb IDs and highlevel data

In [4]:
query = {
  "query": {
    "type": "terminal",
    "service": "sequence",
    "parameters": {
      "evalue_cutoff": 1,
      "identity_cutoff": 0.9,
      "target": "pdb_protein_sequence",
      "value": seq
    }
  },
  "request_options": {
    "scoring_strategy": "sequence"
  },
  "return_type": result_type
}

In [5]:
query_string = json.dumps(query, indent=4)
print(query_string)

url = "https://search.rcsb.org/rcsbsearch/v1/query?json="
response = requests.post(url, data=query_string)

print("\n", response)

{
    "query": {
        "type": "terminal",
        "service": "sequence",
        "parameters": {
            "evalue_cutoff": 1,
            "identity_cutoff": 0.9,
            "target": "pdb_protein_sequence",
            "value": "TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG"
        }
    },
    "request_options": {
        "scoring_strategy": "sequence"
    },
    "return_type": "polymer_entity"
}

 <Response [200]>


In [6]:
hits = json.loads(response.text)
hits_pretty = json.dumps(hits, indent=4)
print(hits_pretty)

{
    "query_id": "bfc3b3cc-1815-449f-b989-04b2ef48e441",
    "result_type": "polymer_entity",
    "total_count": 434,
    "explain_meta_data": {
        "total_timing": 93,
        "sort_timing": 1,
        "terminal_node_timings": {
            "28861": 91
        }
    },
    "result_set": [
        {
            "identifier": "7A92_2",
            "score": 1.0,
            "services": [
                {
                    "service_type": "sequence",
                    "nodes": [
                        {
                            "node_id": 28861,
                            "original_score": 424.0,
                            "norm_score": 1.0,
                            "match_context": [
                                {
                                    "sequence_identity": 1.0,
                                    "evalue": 2.478e-135,
                                    "bitscore": 424,
                                    "alignment_length": 194,
                      

In [7]:
# get the pdb IDs
print(type(hits))
results = hits['result_set']
print(results[0]['identifier'])
print(results[0]['services'][0]['nodes'][0]['match_context'][0]['sequence_identity'])
print(results[0]['services'][0]['nodes'][0]['match_context'][0]['alignment_length'])
print(results[0]['services'][0]['nodes'][0]['match_context'][0]['query_length'])

<class 'dict'>
7A92_2
1.0
194
194


In [8]:
# all values
for hit in hits['result_set']:
    id        = hit['identifier'].split("_")[0]
    bit_score = hit['services'][0]['nodes'][0]['match_context'][0]['bitscore']
    pct_match = round(hit['services'][0]['nodes'][0]['match_context'][0]['sequence_identity'], 2)
    align_len = hit['services'][0]['nodes'][0]['match_context'][0]['alignment_length']
    seq_len   = hit ['services'][0]['nodes'][0]['match_context'][0]['query_length']
    pct_align = round(align_len/seq_len, 2)
    print(id, bit_score, pct_match, align_len, seq_len, pct_align)

7A92 424 1.0 194 194 1.0
7C01 424 1.0 194 194 1.0
7LCN 424 1.0 194 194 1.0
7KZB 424 1.0 194 194 1.0
7CHE 424 1.0 194 194 1.0
7K90 424 1.0 194 194 1.0
7E7Y 424 1.0 194 194 1.0
7LOP 424 1.0 194 194 1.0
7CAK 424 1.0 194 194 1.0
7CH5 424 1.0 194 194 1.0


In [40]:
# Get a file
url = "https://files.rcsb.org/download/"
# file = id + ".pdb" # last one from the above loop
file = results[0]['identifier'].split("_")[0] + ".pdb" # or top score
response = requests.get(url + file)
pdb = response.text

print(response.url)
print(response, "\n")
print(pdb)

https://files.rcsb.org/download/7A92.pdb
<Response [200]> 

HEADER    VIRAL PROTEIN                           01-SEP-20   7A92              
TITLE     DISSOCIATED S1 DOMAIN OF SARS-COV-2 SPIKE BOUND TO ACE2 (UNMASKED     
TITLE    2 REFINEMENT)                                                          
COMPND    MOL_ID: 1;                                                            
COMPND   2 MOLECULE: ANGIOTENSIN-CONVERTING ENZYME 2;                           
COMPND   3 CHAIN: D;                                                            
COMPND   4 SYNONYM: ANGIOTENSIN-CONVERTING ENZYME HOMOLOG,ACEH,ANGIOTENSIN-     
COMPND   5 CONVERTING ENZYME-RELATED CARBOXYPEPTIDASE,ACE-RELATED               
COMPND   6 CARBOXYPEPTIDASE,METALLOPROTEASE MPROT15;                            
COMPND   7 EC: 3.4.17.23,3.4.17.-;                                              
COMPND   8 ENGINEERED: YES;                                                     
COMPND   9 MOL_ID: 2;                            

In [10]:
# an alternative using urllib to encode the json string with http escape characters
#import urllib.parse

#encoded = urllib.parse.quote(query_string)
#print(encoded)
#pdbquery = urllib.request.urlopen('https://search.rcsb.org/rcsbsearch/v1/query?json=' + encoded)

In [72]:
import re
charpts = ([[a.start(),a.end()] for a in list(re.finditer('CHAIN', pdb))])
print(charpts)


m = re.compile(r'.*MOL_ID.*\s', re.MULTILINE)
lines = m.findall(pdb)
print(lines)
print('\n'.join(map(str,lines)))

[[416, 421], [1064, 1069], [11293, 11298], [11886, 11891], [23862, 23867], [27750, 27755]]
['COMPND    MOL_ID: 1;                                                            \n', 'COMPND   9 MOL_ID: 2;                                                           \n', 'SOURCE    MOL_ID: 1;                                                            \n', 'SOURCE   9 MOL_ID: 2;                                                           \n']
COMPND    MOL_ID: 1;                                                            

COMPND   9 MOL_ID: 2;                                                           

SOURCE    MOL_ID: 1;                                                            

SOURCE   9 MOL_ID: 2;                                                           

